In [3]:
API_KEY = "F97yP8gOkQjyt3c9Hvca8VOe745yEAMMLdaNvyFR"
import requests
url = f"https://api.nasa.gov/neo/rest/v1/feed?start_date=2025-01-01&end_date=2025-01-07&api_key={API_KEY}"
response=requests.get(url)
response

from datetime import datetime
import requests

asteroids_data=[]
target=10000

url = f"https://api.nasa.gov/neo/rest/v1/feed?start_date=2025-01-01&end_date=2025-01-07&api_key={API_KEY}"

response=requests.get(url)
data=response.json()
details=data['near_earth_objects']
while len(asteroids_data)<target:
  for date,ast_details in details.items():
    for ast in ast_details:
      asteroids_data.append(dict(
          id=int(ast['id']),
          neo_reference_id=ast['neo_reference_id'],
          name=ast['name'],
          absolute_magnitude_h=float(ast['absolute_magnitude_h']),
          estimated_diameter_min=float(ast['estimated_diameter']['kilometers']['estimated_diameter_min']),
          estimated_diameter_max=float(ast['estimated_diameter']['kilometers']['estimated_diameter_max']),
          is_potentially_hazardous_asteroid=ast['is_potentially_hazardous_asteroid'],
          close_approach_date=datetime.strptime(ast['close_approach_data'][0]['close_approach_date'], "%Y-%m-%d"),
          relative_velocity=float(ast['close_approach_data'][0]['relative_velocity']['kilometers_per_second']),
          astronomical=float(ast['close_approach_data'][0]['miss_distance']['astronomical']),
          miss_distance_km=float(ast['close_approach_data'][0]['miss_distance']['kilometers']),
          miss_distance_lunar=float(ast['close_approach_data'][0]['miss_distance']['lunar']),
          orbiting_body=ast['close_approach_data'][0]['orbiting_body']
      ))
      if len(asteroids_data)==target:
        break
  if len(asteroids_data)==target:
    break
url=data['links']['next']

In [4]:
asteroids_data

[{'id': 2226514,
  'neo_reference_id': '2226514',
  'name': '226514 (2003 UX34)',
  'absolute_magnitude_h': 20.14,
  'estimated_diameter_min': 0.2492039814,
  'estimated_diameter_max': 0.5572370428,
  'is_potentially_hazardous_asteroid': True,
  'close_approach_date': datetime.datetime(2025, 1, 7, 0, 0),
  'relative_velocity': 16.5489937942,
  'astronomical': 0.1236081667,
  'miss_distance_km': 18491518.45292493,
  'miss_distance_lunar': 48.0835768463,
  'orbiting_body': 'Earth'},
 {'id': 2438017,
  'neo_reference_id': '2438017',
  'name': '438017 (2003 YO3)',
  'absolute_magnitude_h': 18.54,
  'estimated_diameter_min': 0.5206609142,
  'estimated_diameter_max': 1.1642331974,
  'is_potentially_hazardous_asteroid': False,
  'close_approach_date': datetime.datetime(2025, 1, 7, 0, 0),
  'relative_velocity': 15.2767515283,
  'astronomical': 0.0811263309,
  'miss_distance_km': 12136326.303555183,
  'miss_distance_lunar': 31.5581427201,
  'orbiting_body': 'Earth'},
 {'id': 2481442,
  'neo_ref

In [6]:
!pip install pymysql
import pymysql

conn = pymysql.connect(
    host='localhost',
    user='root',
    password='Vikram@3ms',
    db='asteroids'
)

In [7]:
import requests
import pymysql
from datetime import datetime

# ---------- CONFIG ----------
API_KEY = "F97yP8gOkQjyt3c9Hvca8VOe745yEAMMLdaNvyFR"
DB_NAME = "nasa_data"

# ---------- CONNECT TO MYSQL ----------
conn = pymysql.connect(
    host="localhost",
    user="root",
    password="Vikram@3ms",
    db="asteroids",
    charset="utf8mb4",
    cursorclass=pymysql.cursors.DictCursor
)
cursor = conn.cursor()

# ---------- CREATE TABLES ----------
cursor.execute("""
CREATE TABLE IF NOT EXISTS asteroids (
    asteroid_pk INT AUTO_INCREMENT PRIMARY KEY,
    id INT,
    name VARCHAR(255),
    absolute_magnitude_h FLOAT,
    estimated_diameter_min_km FLOAT,
    estimated_diameter_max_km FLOAT,
    is_potentially_hazardous_asteroid BOOLEAN
)
""")

cursor.execute("""
CREATE TABLE IF NOT EXISTS close_approach (
    approach_pk INT AUTO_INCREMENT PRIMARY KEY,
    neo_reference_id INT,
    close_approach_date DATE,
    relative_velocity_kmph FLOAT,
    astronomical FLOAT,
    miss_distance_km FLOAT,
    miss_distance_lunar FLOAT,
    orbiting_body VARCHAR(50)
)
""")

conn.commit()
print("Tables created successfully!")

# ---------- FETCH DATA FROM NASA API ----------
url = f"https://api.nasa.gov/neo/rest/v1/feed?start_date=2025-01-01&end_date=2025-01-07&api_key={API_KEY}"
response = requests.get(url)
data = response.json()
details = data["near_earth_objects"]

# ---------- INSERT DATA ----------
for date, ast_list in details.items():
    for ast in ast_list:
        
        # Insert into asteroids table
        cursor.execute("""
            INSERT INTO asteroids
            (id, name, absolute_magnitude_h, estimated_diameter_min_km, estimated_diameter_max_km, is_potentially_hazardous_asteroid)
            VALUES (%s,%s,%s,%s,%s,%s)
        """, (
            int(ast["id"]),
            ast["name"],
            float(ast["absolute_magnitude_h"]),
            float(ast["estimated_diameter"]["kilometers"]["estimated_diameter_min"]),
            float(ast["estimated_diameter"]["kilometers"]["estimated_diameter_max"]),
            ast["is_potentially_hazardous_asteroid"]
        ))

        # Insert into close_approach table
        ca_data = ast["close_approach_data"][0]
        cursor.execute("""
            INSERT INTO close_approach
            (neo_reference_id, close_approach_date, relative_velocity_kmph, astronomical, miss_distance_km, miss_distance_lunar, orbiting_body)
            VALUES (%s,%s,%s,%s,%s,%s,%s)
        """, (
            int(ast["id"]),
            datetime.strptime(ca_data["close_approach_date"], "%Y-%m-%d").date(),
            float(ca_data["relative_velocity"]["kilometers_per_hour"]),
            float(ca_data["miss_distance"]["astronomical"]),
            float(ca_data["miss_distance"]["kilometers"]),
            float(ca_data["miss_distance"]["lunar"]),
            ca_data["orbiting_body"]
        ))

# Commit changes
conn.commit()
print("Data inserted successfully!")

# ---------- CLOSE CONNECTION ----------
cursor.close()
conn.close()


Tables created successfully!
Data inserted successfully!


In [9]:
!pip install streamlit

In [10]:
import streamlit as st

st.title("This is a title")
st.title("_Streamlit_ is :blue[cool] :sunglasses:")

2025-08-15 20:33:36.774 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-08-15 20:33:37.071 
  command:

    streamlit run C:\Users\Welcome\AppData\Local\Programs\Python\Python313\Lib\site-packages\ipykernel_launcher.py [ARGUMENTS]
2025-08-15 20:33:37.073 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-08-15 20:33:37.074 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-08-15 20:33:37.075 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-08-15 20:33:37.076 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-08-15 20:33:37.079 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.


DeltaGenerator()

In [3]:
# asteroid_dashboard.py
import streamlit as st
import pandas as pd
import pymysql

# -----------------------
# DATABASE CONNECTION
# -----------------------
def get_connection():
    return pymysql.connect(
        host="localhost",
    user="root",
    password="Vikram@3ms",
    db="asteroids",
    charset="utf8mb4",
    cursorclass=pymysql.cursors.DictCursor
    )

# -----------------------
# RUN SQL QUERY
# -----------------------
def run_query(query, params=None):
    conn = get_connection()
    try:
        with conn.cursor() as cursor:
            cursor.execute(query, params or ())
            return cursor.fetchall()
    finally:
        conn.close()

# -----------------------
# PREDEFINED QUERIES
# -----------------------
predefined_queries = {
    "All Asteroids": "SELECT * FROM asteroids LIMIT 50;",
    "Potentially Hazardous": "SELECT * FROM asteroids WHERE is_potentially_hazardous_asteroid = 1;",
    "Largest Diameter": "SELECT * FROM asteroids ORDER BY estimated_diameter_max_km DESC LIMIT 10;",
    "Smallest Diameter": "SELECT * FROM asteroids ORDER BY estimated_diameter_min_km ASC LIMIT 10;",
    "Fastest Relative Velocity": """
        SELECT a.name, c.relative_velocity_kmph
        FROM close_approach c
        JOIN asteroids a ON a.id = c.neo_reference_id
        ORDER BY c.relative_velocity_kmph DESC LIMIT 10;
    """,
    # Add more queries here up to 15
}

# -----------------------
# STREAMLIT UI
# -----------------------
st.set_page_config(page_title="🚀 Asteroid Data Explorer", layout="wide")
st.title("🚀 Asteroid Data Explorer")
st.markdown("Explore asteroid close-approach data from NASA's NEO API with dynamic SQL queries.")

# Sidebar: Query selection
st.sidebar.header("🔍 Select a Predefined Query")
query_choice = st.sidebar.selectbox("Choose Query", list(predefined_queries.keys()))
if st.sidebar.button("Run Query"):
    data = run_query(predefined_queries[query_choice])
    st.dataframe(pd.DataFrame(data), use_container_width=True)

st.sidebar.markdown("---")

# -----------------------
# FILTER SECTION
# -----------------------
st.sidebar.header("🎯 Apply Filters")

# Date filter
date_filter = st.sidebar.date_input("Close Approach Date")

# Astronomical Units
au_min, au_max = st.sidebar.slider("Astronomical Units (AU)", 0.0, 5.0, (0.0, 5.0), 0.01)

# Lunar Distances
lunar_min, lunar_max = st.sidebar.slider("Lunar Distances", 0.0, 50.0, (0.0, 50.0), 0.5)

# Relative Velocity
vel_min, vel_max = st.sidebar.slider("Relative Velocity (km/h)", 0.0, 150000.0, (0.0, 150000.0), 100.0)

# Estimated Diameter
diam_min, diam_max = st.sidebar.slider("Estimated Diameter (km)", 0.0, 50.0, (0.0, 50.0), 0.1)

# Hazardous state
hazardous = st.sidebar.selectbox("Potentially Hazardous?", ["All", "Yes", "No"])

# -----------------------
# SQL WITH FILTERS
# -----------------------
query = """
SELECT a.id, a.name, a.estimated_diameter_min_km, a.estimated_diameter_max_km,
       a.is_potentially_hazardous_asteroid, c.close_approach_date,
       c.relative_velocity_kmph, c.astronomical, c.miss_distance_lunar
FROM asteroids a
JOIN close_approach c ON a.id = c.neo_reference_id
WHERE c.close_approach_date = %s
AND c.astronomical BETWEEN %s AND %s
AND c.miss_distance_lunar BETWEEN %s AND %s
AND c.relative_velocity_kmph BETWEEN %s AND %s
AND a.estimated_diameter_min_km >= %s
AND a.estimated_diameter_max_km <= %s
"""

params = [
    date_filter,
    au_min, au_max,
    lunar_min, lunar_max,
    vel_min, vel_max,
    diam_min, diam_max
]

if hazardous == "Yes":
    query += " AND a.is_potentially_hazardous_asteroid = 1"
elif hazardous == "No":
    query += " AND a.is_potentially_hazardous_asteroid = 0"

if st.sidebar.button("Apply Filters"):
    filtered_data = run_query(query, params)
    st.subheader("📊 Filtered Results")
    st.dataframe(
        pd.DataFrame(filtered_data),
        use_container_width=True
    )

# -----------------------
# STYLING
# -----------------------
st.markdown("""
    <style>
    .stDataFrame { font-size: 14px; }
    </style>
""", unsafe_allow_html=True)


2025-08-15 22:02:13.203 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-08-15 22:02:13.205 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-08-15 22:02:13.369 
  command:

    streamlit run C:\Users\Welcome\AppData\Local\Programs\Python\Python313\Lib\site-packages\ipykernel_launcher.py [ARGUMENTS]
2025-08-15 22:02:13.370 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-08-15 22:02:13.371 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-08-15 22:02:13.372 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-08-15 22:02:13.373 Thread 'MainThread': missing ScriptRunContext! This 

DeltaGenerator()

In [4]:
# -----------------------
# PREDEFINED QUERIES
# -----------------------
predefined_queries = {
    "All Asteroids (First 50)": """
        SELECT * FROM asteroids LIMIT 50;
    """,
    "Potentially Hazardous Asteroids": """
        SELECT * FROM asteroids 
        WHERE is_potentially_hazardous_asteroid = 1
        LIMIT 50;
    """,
    "Non-Hazardous Asteroids": """
        SELECT * FROM asteroids 
        WHERE is_potentially_hazardous_asteroid = 0
        LIMIT 50;
    """,
    "Top 10 Largest Diameter": """
        SELECT name, estimated_diameter_max_km
        FROM asteroids
        ORDER BY estimated_diameter_max_km DESC
        LIMIT 10;
    """,
    "Top 10 Smallest Diameter": """
        SELECT name, estimated_diameter_min_km
        FROM asteroids
        ORDER BY estimated_diameter_min_km ASC
        LIMIT 10;
    """,
    "Fastest Relative Velocity (Top 10)": """
        SELECT a.name, c.relative_velocity_kmph
        FROM close_approach c
        JOIN asteroids a ON a.id = c.neo_reference_id
        ORDER BY c.relative_velocity_kmph DESC
        LIMIT 10;
    """,
    "Slowest Relative Velocity (Top 10)": """
        SELECT a.name, c.relative_velocity_kmph
        FROM close_approach c
        JOIN asteroids a ON a.id = c.neo_reference_id
        ORDER BY c.relative_velocity_kmph ASC
        LIMIT 10;
    """,
    "Closest Approach by Lunar Distance": """
        SELECT a.name, c.miss_distance_lunar
        FROM close_approach c
        JOIN asteroids a ON a.id = c.neo_reference_id
        ORDER BY c.miss_distance_lunar ASC
        LIMIT 10;
    """,
    "Farthest Approach by Lunar Distance": """
        SELECT a.name, c.miss_distance_lunar
        FROM close_approach c
        JOIN asteroids a ON a.id = c.neo_reference_id
        ORDER BY c.miss_distance_lunar DESC
        LIMIT 10;
    """,
    "Count of Asteroids per Date": """
        SELECT close_approach_date, COUNT(*) AS asteroid_count
        FROM close_approach
        GROUP BY close_approach_date
        ORDER BY asteroid_count DESC;
    """,
    "Average Diameter per Hazardous State": """
        SELECT is_potentially_hazardous_asteroid,
               AVG(estimated_diameter_max_km) AS avg_max_diameter
        FROM asteroids
        GROUP BY is_potentially_hazardous_asteroid;
    """,
    "Asteroids with Diameter > 10 km": """
        SELECT name, estimated_diameter_max_km
        FROM asteroids
        WHERE estimated_diameter_max_km > 10;
    """,
    "Asteroids Approaching in Next 7 Days": """
        SELECT a.name, c.close_approach_date
        FROM close_approach c
        JOIN asteroids a ON a.id = c.neo_reference_id
        WHERE c.close_approach_date BETWEEN CURDATE() AND DATE_ADD(CURDATE(), INTERVAL 7 DAY)
        ORDER BY c.close_approach_date ASC;
    """,
    "Asteroids within 1 Lunar Distance": """
        SELECT a.name, c.miss_distance_lunar
        FROM close_approach c
        JOIN asteroids a ON a.id = c.neo_reference_id
        WHERE c.miss_distance_lunar <= 1
        ORDER BY c.miss_distance_lunar ASC;
    """,
    "Top 10 Fastest Hazardous Asteroids": """
        SELECT a.name, c.relative_velocity_kmph
        FROM close_approach c
        JOIN asteroids a ON a.id = c.neo_reference_id
        WHERE a.is_potentially_hazardous_asteroid = 1
        ORDER BY c.relative_velocity_kmph DESC
        LIMIT 10;
    """
}


In [5]:
def style_dataframe(df):
    return (
        df.style
        .set_table_styles([
            {'selector': 'thead th',
             'props': [('background-color', '#0b3d91'), ('color', 'white'), ('font-weight', 'bold'), ('text-align', 'center')]},
            {'selector': 'tbody td',
             'props': [('text-align', 'center')]},
            {'selector': 'tbody tr:nth-child(even)',
             'props': [('background-color', '#f2f2f2')]},
            {'selector': 'tbody tr:nth-child(odd)',
             'props': [('background-color', 'white')]},
        ])
        .set_properties(**{'border': '1px solid #ccc', 'padding': '6px'})
    )
